In [1]:
import json

In [4]:
# load new json file
with open('C:/Users/sucho/OneDrive - UCB-O365/Desktop/Research projects/SCI/schema-interface-rs/static/SF3template.json') as f:
    new_schema_format = json.load(f)

# load old json file
with open('C:/Users/sucho/OneDrive - UCB-O365/Desktop/Research projects/SCI/schema-interface-rs/static/restaurant_schema.json') as f:
    old_schema_format = json.load(f)

In [10]:
schema_key_dict = {
    'event': ['@id', 'name', 'comment', 'description', 'aka', 'qnode', 'qlabel', 'isSchema', 'goal', 'ta1explanation', 'importance', 'children_gate', 'instanceOf', 'probParent', 'probChild', 'probability', 'liklihood', 'wd_node', 'wd_label', 'wd_description', 'modality', 'participants', 'privateData', 'outlinks', 'entities', 'relations', 'children', 'optional', 'repeatable'],
    'children': ['child', 'comment', 'optional', 'importance', 'outlinks'],
    'privateData': ['@type', 'template', 'repeatable', 'importance'],
    'entity': ['name', '@id', 'qnode', 'qlabel', 'centrality', 'wd_node', 'wd_label', 'wd_description', 'modality', 'aka','properties'],
    'properties': ['property values'],
    'relation': ['name', 'wd_node', 'wd_label', 'modality', 'wd_description', 'ta1ref', 'relationSubject', 'relationObject', 'relationPredicate']
}

In [16]:
def update_json(values):
    """Updates JSON with values.

    Parameters:
    values (dict): contains node id, key, and value to change key to.
    e.g. {id: node_id, key: name, value: Test}

    Returns:
    schemaJson (dict): new JSON 
    """
    global schema_json
    new_json = schema_json
    node_id = values['id']
    node_type = False
    print("Received Values: ", values)
    print("node_id: ", node_id)
    key = values['key']
    new_value = values['value']
    if key in ['source', 'target']:
        node_type = 'edge'
        return new_json
    else:
        node_type = node_id.split('/')[0].split(':')[-1].lower()
    is_root = node_id == 'Events/10000/'

    # TODO how to edit relations and participants through the sidebar?

    # entities
    if node_type == 'entities':
        # entity data
        for entity in new_json['events']['entities']:
            if entity['@id'] == node_id:
                entity[key] = new_value
        if key != '@id':
            schema_json = new_json
            return schema_json
        else:
            # relation data
            for relation in new_json['events']['relations']:
                if relation['relationSubject'] == node_id:
                    relation['relationSubject'] = new_value
                if relation['relationObject'] == node_id:
                    relation['relationObject'] = new_value
                pass

    # nodes
    # child key
    if key == 'name':
        child_key = 'comment'
    elif key == '@id':
        child_key = 'child'
    else:
        child_key = key

    for scheme in new_json['events']:
        # entity id search
        if node_type == 'entities':
            if 'participants' in scheme:
                for participant in scheme['participants']:
                    if participant['entity'] == node_id:
                        participant['entity'] = new_value
                        # if entity is blank, add 'Entities/20000/'
                        if participant['entity'] == '':
                            participant['entity'] = 'Entities/20000/'

        else:
            # scheme data
            if scheme['@id'] == node_id:
                if key in scheme:
                    scheme[key] = new_value
                    if key == 'comment':
                        break
                    if key in schema_key_dict['event'] or is_root:
                        break
                elif key in schema_key_dict['privateData'] and 'privateData' in scheme:
                    if key in scheme['privateData']:
                        scheme['privateData'][key] = new_value
                        break
            # children data
            if 'children' in scheme and child_key in schema_key_dict['child']:
                for child in scheme['children']:
                    # child
                    if child['child'] == node_id:
                        child[child_key] = new_value
                    # child outlinks
                    if child_key == 'child':
                        for i in range(len(child['outlinks'])):
                            if child['outlinks'][i] == node_id:
                                child['outlinks'][i] = new_value
            # participant data is not listed in sidebar

    schema_json = new_json
    return schema_json

In [17]:
update_json(old_schema_format)

NameError: name 'schema_json' is not defined

In [8]:
# Now, we need to create a function which will transform the new schema format to the old schema format
def transform_schema(new_schema_format, old_schema_format):
    # create a new dictionary to store the transformed schema

